In [1]:
import numpy as np
from tqdm import tqdm
import torch
import glob
from PIL import Image
import re

In [13]:
file = r"/home/smitesh22/Data/DEM/N21E074_wgs84.tif"

In [14]:
im = Image.open(file)

In [15]:
data = np.asanyarray(im)

In [ ]:
import numpy as np

def calculate_slope(dem_matrix, resolution):
    # Get the dimensions of the DEM matrix
    rows, cols = dem_matrix.shape
    
    # Create a new matrix to store the slope values
    slope_matrix = np.zeros((rows, cols))
    
    # Iterate over each cell in the DEM matrix
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Calculate the height differences between neighboring cells
            delta_z_x = dem_matrix[i, j+1] - dem_matrix[i, j-1]
            delta_z_y = dem_matrix[i+1, j] - dem_matrix[i-1, j]
            
            # Calculate the horizontal and vertical distances
            distance_x = 2 * resolution
            distance_y = 2 * resolution
            
            # Calculate the slope using the formula: slope = arctan(delta z / distance)
            slope_x = np.arctan(delta_z_x / distance_x)
            slope_y = np.arctan(delta_z_y / distance_y)
            
            # Calculate the overall slope magnitude
            slope = np.sqrt(slope_x**2 + slope_y**2)
            
            # Assign the slope value to the corresponding cell in the slope matrix
            slope_matrix[i, j] = slope
    
    return slope_matrix

# Example usage
# Assuming you have your DEM data matrix stored in a variable called "dem_data"
resolution = 30  # Spatial resolution in meters

# Call the function to calculate the slope
slope_data = calculate_slope(data, resolution)

# The resulting slope_data will contain the slope values for each cell in the DEM matrix.


In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [60]:
file_paths = glob.glob('/home/smitesh22/Data/DEM/*.tif')

progress_bar = tqdm(total=len(file_paths))
pattern = r'\/(\w+)\.tif$'
dem_dict = {}

for i, location in enumerate(file_paths):
    image = np.asanyarray(Image.open(location))
    out = calculate_slope(image, 30)
    out = torch.tensor(out)
    
    match = re.search(pattern, location)
    
    dem_dict[match.group(1)] = out
    progress_bar.update(1)






  6%|███████▏                                                                                                                | 3/50 [01:09<18:08, 23.17s/it]



  2%|██▎                                                                                                                   | 1/50 [01:30<1:13:59, 90.61s/it]


  4%|████▋                                                                                                                 | 2/50 [03:12<1:17:55, 97.41s/it]


  6%|███████                                                                                                              | 3/50 [04:57<1:18:51, 100.66s/it]


  8%|█████████▎                                                                                                           | 4/50 [06:44<1:19:13, 103.35s/it]


 10%|███████████▋                                                                                                         | 5/50 [08:30<1:18:09, 104.21s/it]


 12%|██████████████                       

In [61]:
torch.save(dem_dict, '/home/smitesh22/Data/DEM_slope_tensor.pt')